In [2]:
!pip install numpy
!pip install time
!pip install datetime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 36.0 MB/s eta 0:00:0000:010:01
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [datetime]2/4 [zope.interface]


In [13]:
import numpy as np
import time
from datetime import datetime, timedelta
import random
import csv
import os
import threading

def log_state(log_file, state):
    """Thread-safe logging function"""
    with open(log_file, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now().strftime('%Y-%m-%d %H:%M:%S'), state])

# Set up the run times
total_runtime = timedelta(minutes=110)
initial_normal_period = timedelta(minutes=15)
start_time = datetime.now()

# List of matrix sizes
normal_sizes = [1800, 2000, 2200, 2400]
anomaly_sizes = [1000, 3000]

# Create or open the CSV file for logging
log_file = 'execution_log.csv'
with open(log_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp', 'state'])  # Write header

# Create a flag for the logging thread
continue_logging = True

def continuous_logging():
    """Function to continuously log the current state"""
    last_log_time = datetime.now()
    while continue_logging:
        current_time = datetime.now()
        # Log every second
        if (current_time - last_log_time).total_seconds() >= 1:
            log_state(log_file, current_state)
            last_log_time = current_time
        time.sleep(0.1)  # Small sleep to prevent excessive CPU usage

# Start the logging thread
current_state = 'normal'
logging_thread = threading.Thread(target=continuous_logging, daemon=True)
logging_thread.start()

try:
    # Initial 15-minute normal period
    while datetime.now() - start_time < initial_normal_period:
        matrix_size = random.choice(normal_sizes)
        
        # CPU intensive operation
        matrix_a = np.random.rand(matrix_size, matrix_size)
        matrix_b = np.random.rand(matrix_size, matrix_size)
        
        for _ in range(5):
            result = np.dot(matrix_a, matrix_b)
            matrix_a = result / np.linalg.norm(result)
        
        del result, matrix_a, matrix_b
        time.sleep(4)

    # Continue with the normal/anomaly pattern for the remaining time
    while datetime.now() - start_time < total_runtime:
        random_choice = random.randint(1, 20)
        
        # Update the current state based on the random choice
        if random_choice == 20:
            matrix_size = random.choice(anomaly_sizes)
            current_state = 'anomaly'
        else:
            matrix_size = random.choice(normal_sizes)
            current_state = 'normal'
        
        # CPU intensive operation
        matrix_a = np.random.rand(matrix_size, matrix_size)
        matrix_b = np.random.rand(matrix_size, matrix_size)
        
        for _ in range(5):
            result = np.dot(matrix_a, matrix_b)
            matrix_a = result / np.linalg.norm(result)
        
        del result, matrix_a, matrix_b
        
        # Reset state to normal during sleep
        current_state = 'normal'
        time.sleep(4)

finally:
    # Clean up the logging thread
    continue_logging = False
    logging_thread.join(timeout=1)
    print("Script completed")

Script completed


KeyboardInterrupt: 